In [7]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime
import requests

In [5]:
params = {
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

data=params

In [8]:
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_PRICES
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
items_id = fetch_items_from_storage(storage, settings.BUCKET_STORES, blob_items_prefix, 'id')
print(f'** Items found: {len(items_id)}**')
print(f'** Cleaning blob **')
# Path for saving price details
blob_basic_path = settings.BLOB_PRICES(store_name)
date_blob_path = f'{blob_basic_path}date={today_str}/'
# Clean existing files in the storage bucket
storage.clean_blobs(bucket_name, date_blob_path)
print(f'** Starting API requests for {len(items_id)} items**')
# URL function for API
url = settings.URL_PRICE
url

** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=0__total_items=100__process_time=2024-09-23T10:15:15.156411-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=1__total_items=100__process_time=2024-09-23T10:15:15.748949-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=2__total_items=100__process_time=2024-09-23T10:15:15.397569-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=3__total_items=100__process_time=2024-09-23T10:15:15.991552-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_id=4__

<function src.config.settings.<lambda>(item_id)>

In [16]:
headers = {'Authorization': f'Bearer {access_token}'}

In [47]:
response = requests.get(url('MLB3348592687'), headers=headers)
json = response.json()
json

{'id': 'MLB3348592687',
 'prices': [{'id': '26',
   'type': 'standard',
   'amount': 369,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2023-11-09T13:54:49Z',
   'conditions': {'context_restrictions': [],
    'start_time': None,
    'end_time': None}},
  {'id': '243',
   'type': 'promotion',
   'amount': 287.82,
   'regular_amount': 369.0,
   'currency_id': 'BRL',
   'last_updated': '2024-09-16T12:41:24Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': None,
    'end_time': None}},
  {'id': '244',
   'type': 'promotion',
   'amount': 287.82,
   'regular_amount': 369.0,
   'currency_id': 'BRL',
   'last_updated': '2024-09-16T12:41:24Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': None,
    'end_time': None}},
  {'id': '245',
   'type': 'promotion',
   'amount': 284.13,
   'regular_amount': 369.0,
   'currency_id': 'BRL',
   'last_updated': '2024-09-16T12:41:24Z',
   'conditions': {'c

In [128]:
extracted_data = []
# Dicionário temporário para priorizar os preços por canal
price_by_channel = {}
for price in json['prices']:
    channel = price['conditions']['context_restrictions']
    if len(channel) == 1:
        channel = channel[0]
        # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
        if channel not in price_by_channel or price['type'] == 'promotion':
            price_by_channel[channel] = {
                'item_id': json.get('id'),
                'price_id': price.get('id'),
                'regular_amount': price.get('regular_amount'),
                'price': price.get('amount'),
                'channel': channel,
                'last_updated': price.get('last_updated')
            }
    else:
        channel = None
        if price_by_channel=={}:
            price_by_channel[channel] = {
                    'item_id': json.get('id'),
                    'price_id': price.get('id'),
                    'regular_amount': price.get('regular_amount'),
                    'price': price.get('amount'),
                    'channel': channel,
                    'last_updated': price.get('last_updated')
                }

# Converte os valores armazenados para uma lista
extracted_data.extend(price_by_channel.values())
extracted_data

[{'item_id': 'MLB5000922536',
  'price_id': '4',
  'regular_amount': 21.3,
  'price': 17.04,
  'channel': 'channel_marketplace',
  'last_updated': '2024-09-11T21:52:11Z'},
 {'item_id': 'MLB5000922536',
  'price_id': '3',
  'regular_amount': None,
  'price': 37.25,
  'channel': 'channel_mshops',
  'last_updated': '2024-09-11T21:39:19Z'}]

In [26]:
price_by_channel == {}

True

In [49]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

In [50]:
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_COSTS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Getting params to see costs
query = f'''
    with items_details as (
    select distinct
        item_id,
        listing_type,
        category_id
    from datalake-v2-424516.datalake_v2.items_details
    where
        1=1
        and date(correspondent_date) = current_date()
        and seller_id = {seller_id}
    )
    select 
    p.item_id as id,
    d.listing_type as listing_type_id,
    d.category_id,
    p.price,
    p.channel
    from datalake-v2-424516.datalake_v2.items_prices p
    inner join items_details d
    on p.item_id = d.item_id
    where 
    1=1
    and date(p.correspondent_date) = current_date()
'''
# blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
# items_id = fetch_items_from_storage(
# storage, 
# bucket_name, 
# blob_items_prefix, 
# key_names=['id','price', 'category_id', 'listing_type_id']
# )
df_params = bigquery.run_query(query)
# items = df_params[['id','channel']].to_dict(orient='records')
df_params['channel'] = df_params['channel'].apply(lambda x : x.replace('channel_', '')).drop(columns = 'channel')
items_id = df_params.to_dict(orient='records')
print(f'** Items found: {len(items_id)}**')
print(f'** Cleaning blob **')
# Path for saving 
blob_basic_path = settings.BLOB_COSTS(store_name)
date_blob_path = f'{blob_basic_path}date={today_str}/'
# Clean existing files in the storage bucket
storage.clean_blobs(bucket_name, date_blob_path)
print(f'** Starting API requests for {len(items_id)} items**')
# URL function for API
url = settings.URL_COST
headers = {'Authorization': f'Bearer {access_token}'}

** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
** Items found: 1183**
** Cleaning blob **
All blobs with prefix hubsmarthome/meli/api_response/item_cost/date=2024-09-23/ have been deleted.
** Starting API requests for 1183 items**


In [64]:
response = requests.get(url, headers=headers, params=items_id[0])

data = response.json()

{'currency_id': 'BRL',
 'free_relist': False,
 'listing_exposure': 'highest',
 'listing_fee_amount': 0,
 'listing_fee_details': {'fixed_fee': 0, 'gross_amount': 0},
 'listing_type_id': 'gold_special',
 'listing_type_name': 'Clássico',
 'requires_picture': True,
 'sale_fee_amount': 4.36,
 'sale_fee_details': {'fixed_fee': 0,
  'gross_amount': 4.36,
  'percentage_fee': 1.99},
 'stop_time': '2044-09-18T00:00:00.000-04:00'}

In [79]:
(99 + 1) % 100 == 0

True

In [82]:
my_responses = []
batch_number = 0

for i, item in enumerate(items_id):
    print(f'Iteracao: {i}')
    if (i + 1) % 100 == 0:
        # Storing batch
        process_time = datetime.now().strftime(f"%Y-%m-%dT%H:%M:%M.%f-03:00")
        filename = f'batch_{batch_number}__process_time={process_time}.json'
        storage.upload_json(bucket_name, f'{date_blob_path}{filename}', my_responses)
        print(f'Saving file in storage: {filename}')
        batch_number +=1
        my_responses = []
        print()

    response = requests.get(url, headers=headers, params=item)
    data = response.json()
    data['item_id'] = item
    my_responses.append(data)

log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)

    


Iteracao: 0
Iteracao: 1
Iteracao: 2
Iteracao: 3
Iteracao: 4
Iteracao: 5
Iteracao: 6
Iteracao: 7
Iteracao: 8
Iteracao: 9
Iteracao: 10
Iteracao: 11
Iteracao: 12
Iteracao: 13
Iteracao: 14
Iteracao: 15
Iteracao: 16
Iteracao: 17
Iteracao: 18
Iteracao: 19
Iteracao: 20
Iteracao: 21
Iteracao: 22
Iteracao: 23
Iteracao: 24
Iteracao: 25
Iteracao: 26
Iteracao: 27
Iteracao: 28
Iteracao: 29
Iteracao: 30
Iteracao: 31
Iteracao: 32
Iteracao: 33
Iteracao: 34
Iteracao: 35
Iteracao: 36
Iteracao: 37
Iteracao: 38
Iteracao: 39
Iteracao: 40
Iteracao: 41
Iteracao: 42
Iteracao: 43
Iteracao: 44
Iteracao: 45
Iteracao: 46
Iteracao: 47
Iteracao: 48
Iteracao: 49
Iteracao: 50
Iteracao: 51
Iteracao: 52
Iteracao: 53
Iteracao: 54
Iteracao: 55
Iteracao: 56
Iteracao: 57
Iteracao: 58
Iteracao: 59
Iteracao: 60
Iteracao: 61
Iteracao: 62
Iteracao: 63
Iteracao: 64
Iteracao: 65
Iteracao: 66
Iteracao: 67
Iteracao: 68
Iteracao: 69
Iteracao: 70
Iteracao: 71
Iteracao: 72
Iteracao: 73
Iteracao: 74
Iteracao: 75
Iteracao: 76
Iteracao:

In [83]:
log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)


# Testing insert_details

In [85]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json


In [86]:
def extract_seller_sku(attributes):
    for attribute in attributes:
        if attribute.get('id') == 'SELLER_SKU':
            return attribute.get('value_name')
    return None  


def process_details(json_details, json_variations):

    df_product = pd.DataFrame()
    indice_variations = 0

    for j in range(len(json_details)):

         item = json_details[j]

         # Checking if item has variations
         if extract_seller_sku(item.get('attributes', [])):
             has_variation = False
         else:
             has_variation = True   

         # get general information
         product_details_general = {
             'item_id': item.get('id'),
             'item_name': item.get('title'),
             'seller_id': item.get('seller_id'),
             'category_id': item.get('category_id'),
             'official_store_id': item.get('official_store_id'),
             'price': item.get('price'),
             'base_price': item.get('base_price'),
             'original_price': item.get('original_price'),
             'initial_quantity': item.get('initial_quantity'),
             'status': item.get('status'),
             'listing_type': item.get('listing_type_id'),
             'url': item.get('permalink'),
             'free_shipping': item.get('shipping',{}).get('free_shipping'),
             'catalog_id' : item.get('catalog_product_id'),
             'picture_url': item.get('pictures', [{}])[0].get('url'),
             'catalog_listing': item.get('catalog_listing', ''),
             'item_health': item.get('health',''),
         }  
         # If product does not have variations
         if not has_variation:
             product_detail_variation = {
                 'inventory_id': item.get('inventory_id'),
                 'currency_id': item.get('currency_id'),
                 'stock': item.get('available_quantity'),
                 'sold_quantity': item.get('sold_quantity'),
                 'seller_sku': extract_seller_sku(item.get('attributes', [])),
                 'variation_id': np.nan
             }

             product_details_general.update(product_detail_variation)
             df_ = pd.DataFrame([product_details_general])
             df_product = pd.concat([df_product, df_], ignore_index=True)

         # If product has variations
         else:
             for var in item.get('variations', []):
                 variation = json_variations[indice_variations]
                 variation_id = var['id']

                 product_detail_variation = {
                     'inventory_id': variation.get('inventory_id'),
                     'currency_id': variation.get('currency_id'),
                     'stock': variation.get('available_quantity'),
                     'sold_quantity': variation.get('sold_quantity'),
                     'seller_sku': extract_seller_sku(variation.get('attributes', [])),
                     'variation_id': variation_id
                 }

                 indice_variations += 1

                 product_details_general.update(product_detail_variation)
                 df_ = pd.DataFrame([product_details_general])
                 df_product = pd.concat([df_product, df_], ignore_index=True)  

    return df_product   



In [94]:
data = {
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_DETAILS
blob_details = settings.BLOB_ITEMS_DETAILS(store_name)
blob_variations = settings.BLOB_VARIATIONS(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix_details = blob_details + f'date={date_to_process}/'
    blob_prefix_variations = blob_variations + f'date={date_to_process}/'
    # List all the files
    blobs_details = storage.list_blobs(bucket_name, blob_prefix_details)
    blobs_variations = storage.list_blobs(bucket_name, blob_prefix_variations)
    df_processed_data = pd.DataFrame()
    # Processing each blob
    for i in range(len(blobs_details)):
        # Get content information for details and variations
        print(f"Reading file: {blobs_details[i].name}")
        content_details = storage.download_json(bucket_name, blobs_details[i].name)
        print(f"Reading file: {blobs_variations[i].name}")
        content_variations = storage.download_json(bucket_name, blobs_variations[i].name)
        # Process data
        df_partial_processed_data = process_details(content_details, content_variations)
        # Save all data into a single df
        df_processed_data = pd.concat([df_processed_data, df_partial_processed_data], ignore_index=True)
    df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
    df_processed_data['process_time'] = datetime.now()
    df_processed_data['seller_id'] = seller_id

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 1 dates to process  ***
Processing date: 2024-09-22
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-22/batch_id=0__total_items=100__process_time=2024-09-22T09:10:10.966871-03:00.json
Reading file: hubsmarthome/meli/api_response/variation_detail/date=2024-09-22/batch_id=0__total_variations=49__process_time=2024-09-22T09:10:10.966871-03:00.json


C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)


Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-22/batch_id=1__total_items=100__process_time=2024-09-22T09:10:10.053163-03:00.json
Reading file: hubsmarthome/meli/api_response/variation_detail/date=2024-09-22/batch_id=1__total_variations=56__process_time=2024-09-22T09:10:10.053163-03:00.json


C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)


Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-22/batch_id=2__total_items=100__process_time=2024-09-22T09:10:10.418455-03:00.json
Reading file: hubsmarthome/meli/api_response/variation_detail/date=2024-09-22/batch_id=2__total_variations=44__process_time=2024-09-22T09:10:10.418455-03:00.json


...

C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)


Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-22/batch_id=8__total_items=100__process_time=2024-09-22T09:11:11.782613-03:00.json
Reading file: hubsmarthome/meli/api_response/variation_detail/date=2024-09-22/batch_id=8__total_variations=37__process_time=2024-09-22T09:11:11.782613-03:00.json


C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_10196\361639371.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)


Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-22/batch_id=9__total_items=1__process_time=2024-09-22T09:11:11.254941-03:00.json


IndexError: list index out of range

In [109]:
blobs_details = storage.list_blobs(bucket_name, blob_prefix_details)


In [113]:
blob_prefix_details

'hubsmarthome/meli/api_response/item_detail/date=2024-09-22/'

In [112]:
len(blobs_details)

10

In [102]:
blob_details[8]

'h'

In [98]:
print(f"Reading file: {blobs_variations[i].name}")

IndexError: list index out of range

In [115]:

def process_prices(json):

    try:
        extracted_data = []
        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}
        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]
                # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
                if channel not in price_by_channel or price['type'] == 'promotion':
                    price_by_channel[channel] = {
                        'item_id': json.get('id'),
                        'price_id': price.get('id'),
                        'regular_amount': price.get('regular_amount'),
                        'price': price.get('amount'),
                        'channel': channel,
                        'last_updated': price.get('last_updated')
                    }
            # Se o canal for nulo e nao tiver nenhum outro preço
            else:
                channel = None
                price_by_channel[channel] = {
                        'item_id': json.get('id'),
                        'price_id': price.get('id'),
                        'regular_amount': price.get('regular_amount'),
                        'price': price.get('amount'),
                        'channel': channel,
                        'last_updated': price.get('last_updated')
                    }

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')

In [118]:
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_PRICES
blob_shipping_cost = settings.BLOB_PRICES(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = ['2024-09-23']
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = '2024-09-23'
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)
    # Processing each blob
    for blob in blobs:
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)
        for json in content:
            processed_dict = process_prices(json)
            if isinstance(processed_dict, list):
                df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
            else:
                continue
    df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
    df_processed_data['process_time'] = datetime.now()
    df_processed_data['seller_id'] = seller_id
    print(f'*** Finished treating all data. {df_processed_data.shape[0]} products ***')

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 1 dates to process  ***
Processing date: 2024-09-23
Reading file: hubsmarthome/meli/api_response/item_price/date=2024-09-23/batch_0__process_time=2024-09-23T18:04:04.368787-03:00.json
Reading file: hubsmarthome/meli/api_response/item_price/date=2024-09-23/batch_1__process_time=2024-09-23T18:04:04.384532-03:00.json
Reading file: hubsmarthome/meli/api_response/item_price/date=2024-09-23/batch_2__process_time=2024-09-23T18:04:04.426271-03:00.json
Reading file: hubsmarthome/meli/api_response/item_price/date=2024-09-23/batch_3__process_time=2024-09-23T18:04:04.440682-03:00.json
Reading file: hubsmarthome/meli/api_response/item_price/date=2024-09-23/batch_4__process_time=2024-09-23T18:04:04.54688

In [120]:
process_prices(json)

[]

In [131]:
json

{'id': 'MLB5000922536',
 'prices': [{'id': '2',
   'type': 'standard',
   'amount': 21.3,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2024-09-11T21:39:19Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': None,
    'end_time': None}},
  {'id': '3',
   'type': 'standard',
   'amount': 37.25,
   'regular_amount': None,
   'currency_id': 'BRL',
   'last_updated': '2024-09-11T21:39:19Z',
   'conditions': {'context_restrictions': ['channel_mshops'],
    'start_time': None,
    'end_time': None}},
  {'id': '4',
   'type': 'promotion',
   'amount': 17.04,
   'regular_amount': 21.3,
   'currency_id': 'BRL',
   'last_updated': '2024-09-11T21:52:11Z',
   'conditions': {'context_restrictions': ['channel_marketplace'],
    'start_time': '2024-09-01T03:00:00Z',
    'end_time': '2024-10-01T02:59:59Z'}}],
 'item_id': 'MLB5000922536'}

In [132]:


extracted_data = []
# Dicionário temporário para priorizar os preços por canal
price_by_channel = {}
for price in json['prices']:
    channel = price['conditions']['context_restrictions']
    if len(channel) == 1:
        channel = channel[0]
        # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
        if channel not in price_by_channel or price['type'] == 'promotion':
            price_by_channel[channel] = {
                'item_id': json.get('id'),
                'price_id': price.get('id'),
                'regular_amount': price.get('regular_amount'),
                'price': price.get('amount'),
                'channel': channel,
                'last_updated': price.get('last_updated')
            }
    # Se o canal for nulo e nao tiver nenhum outro preço
    else:
        channel = None
        if channel not in price_by_channel:
            price_by_channel[channel] = {
                    'item_id': json.get('id'),
                    'price_id': price.get('id'),
                    'regular_amount': price.get('regular_amount'),
                    'price': price.get('amount'),
                    'channel': channel,
                    'last_updated': price.get('last_updated')
                }
extracted_data.extend(price_by_channel.values())
extracted_data

[{'item_id': 'MLB5000922536',
  'price_id': '4',
  'regular_amount': 21.3,
  'price': 17.04,
  'channel': 'channel_marketplace',
  'last_updated': '2024-09-11T21:52:11Z'},
 {'item_id': 'MLB5000922536',
  'price_id': '3',
  'regular_amount': None,
  'price': 37.25,
  'channel': 'channel_mshops',
  'last_updated': '2024-09-11T21:39:19Z'}]

In [124]:
price_by_channel

{'channel_marketplace': {'item_id': 'MLB5000922536',
  'price_id': '4',
  'regular_amount': 21.3,
  'price': 17.04,
  'channel': 'channel_marketplace',
  'last_updated': '2024-09-11T21:52:11Z'},
 'channel_mshops': {'item_id': 'MLB5000922536',
  'price_id': '3',
  'regular_amount': None,
  'price': 37.25,
  'channel': 'channel_mshops',
  'last_updated': '2024-09-11T21:39:19Z'}}

In [1]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

In [4]:
params = {
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

data=params

In [19]:
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_DETAILS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'

** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


In [65]:
blobs = storage.list_blobs(bucket_name, blob_items_prefix)
list_ids=[]
for blob in blobs:
    if blob.name.endswith('.json'):
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)
        list_ids += {'variation_id':var['id'], 'item_id':json['id'] for json in content if json['variations'] for var in json['variations']}

Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_0__process_time=2024-09-23T23:15:15.568769-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_1__process_time=2024-09-23T23:15:15.897972-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_2__process_time=2024-09-23T23:15:15.261245-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_3__process_time=2024-09-23T23:15:15.625685-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_4__process_time=2024-09-23T23:16:16.026318-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_5__process_time=2024-09-23T23:16:16.411734-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_6__process_time=2024-09-23T23:16:16.725906-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09

In [68]:
[{'variation_id': var['id'], 'item_id': json['id']} for json in content if json['variations'] for var in json['variations']]

25

In [66]:
json

{'id': 'MLB3636066195',
 'site_id': 'MLB',
 'title': 'Bolinha Inteligente Smart Ekaza Eks-fbi050 Cor Branco',
 'family_name': None,
 'seller_id': 189643563,
 'category_id': 'MLB270089',
 'user_product_id': None,
 'official_store_id': 51354,
 'price': 199,
 'base_price': 199,
 'original_price': None,
 'inventory_id': 'MIXG55584',
 'currency_id': 'BRL',
 'initial_quantity': 10,
 'available_quantity': 10,
 'sold_quantity': 0,
 'sale_terms': [{'id': 'WARRANTY_TIME',
   'name': 'Tempo de garantia',
   'value_id': None,
   'value_name': '12 meses',
   'value_struct': {'number': 12, 'unit': 'meses'},
   'values': [{'id': None,
     'name': '12 meses',
     'struct': {'number': 12, 'unit': 'meses'}}],
   'value_type': 'number_unit'},
  {'id': 'WARRANTY_TYPE',
   'name': 'Tipo de garantia',
   'value_id': '2230279',
   'value_name': 'Garantia de fábrica',
   'value_struct': None,
   'values': [{'id': '2230279',
     'name': 'Garantia de fábrica',
     'struct': None}],
   'value_type': 'list'}]

In [59]:
len(content)

100

In [46]:
[json for json in content][0]


{'id': 'MLB4830808640',
 'site_id': 'MLB',
 'title': 'Novadigital  Clw- Dual+ir Controlador Wifi Inteligente Para Fita De Led Rgb Tuya Alexa Tensão De Operação 12v 110v/220v',
 'family_name': None,
 'seller_id': 189643563,
 'category_id': 'MLB189908',
 'user_product_id': 'MLBU1332653163',
 'official_store_id': 5637,
 'price': 78.48,
 'base_price': 78.48,
 'original_price': 87.2,
 'inventory_id': 'FNAF89918',
 'currency_id': 'BRL',
 'initial_quantity': 54,
 'available_quantity': 0,
 'sold_quantity': 54,
 'sale_terms': [{'id': 'WARRANTY_TYPE',
   'name': 'Tipo de garantia',
   'value_id': '2230279',
   'value_name': 'Garantia de fábrica',
   'value_struct': None,
   'values': [{'id': '2230279',
     'name': 'Garantia de fábrica',
     'struct': None}],
   'value_type': 'list'},
  {'id': 'WARRANTY_TIME',
   'name': 'Tempo de garantia',
   'value_id': None,
   'value_name': '12 meses',
   'value_struct': {'number': 12, 'unit': 'meses'},
   'values': [{'id': None,
     'name': '12 meses',
 

In [50]:
list_ids = []
for json in content:
    if json['variations'] != []:
        for var in json['variations']:
            list_ids.append(var['id'])

len(list_ids)

42

In [53]:
list_ids = [var['id'] for json in content if json['variations'] for var in json['variations']]


42

# Correcting insert details 

In [3]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json

In [1]:
params = {
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [28]:
data = params
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_DETAILS
blob_details = settings.BLOB_ITEMS_DETAILS(store_name)
blob_variations = settings.BLOB_VARIATIONS(store_name)

# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix_details = blob_details + f'date={date_to_process}/'
    blob_prefix_variations = blob_variations + f'date={date_to_process}/'
    # List all the files
    blobs_details = storage.list_blobs(bucket_name, blob_prefix_details)
    blobs_variations = storage.list_blobs(bucket_name, blob_prefix_variations)
    
    # Empty variables
    df_processed_data = pd.DataFrame()
    content_details=[]
    content_variations=[]

    # Getting details data
    for blob_det in blobs_details:
        # Get content information for details and variations
        print(f"Reading file: {blob_det.name}")
        content_details += storage.download_json(bucket_name, blob_det.name)

    # Getting variation data
    for blob_var in blobs_variations:
        print(f"Reading file: {blob_var.name}")
        content_variations += storage.download_json(bucket_name, blob_var.name)

    df_product = process_details(content_details, content_variations)


** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 1 dates to process  ***
Processing date: 2024-09-23
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_0__process_time=2024-09-23T23:15:15.568769-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_1__process_time=2024-09-23T23:15:15.897972-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_2__process_time=2024-09-23T23:15:15.261245-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_3__process_time=2024-09-23T23:15:15.625685-03:00.json
Reading file: hubsmarthome/meli/api_response/item_detail/date=2024-09-23/batch_4__process_time=2024-09-23T23:16:16.

C:\Users\User\AppData\Local\Temp\ipykernel_11444\3130426408.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)
C:\Users\User\AppData\Local\Temp\ipykernel_11444\3130426408.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_product = pd.concat([df_product, df_], ignore_index=True)


In [12]:
def extract_seller_sku(attributes):
    for attribute in attributes:
        if attribute.get('id') == 'SELLER_SKU':
            return attribute.get('value_name')
    return None  

In [25]:
def process_details(content_details, content_variations):  

    df_product = pd.DataFrame()

        # Checking if item has variations
    for item in content_details:

        if extract_seller_sku(item.get('attributes', [])):
            has_variation = False
        else:
            has_variation = True   
        # get general information
        product_details_general = {
            'item_id': item.get('id'),
            'item_name': item.get('title'),
            'seller_id': item.get('seller_id'),
            'category_id': item.get('category_id'),
            'official_store_id': item.get('official_store_id'),
            'price': item.get('price'),
            'base_price': item.get('base_price'),
            'original_price': item.get('original_price'),
            'initial_quantity': item.get('initial_quantity'),
            'status': item.get('status'),
            'listing_type': item.get('listing_type_id'),
            'url': item.get('permalink'),
            'free_shipping': item.get('shipping',{}).get('free_shipping'),
            'picture_url': item.get('pictures', [{}])[0].get('url'),
            'catalog_listing': item.get('catalog_listing', ''),
            'item_health': item.get('health',''),
        }  
        # If product does not have variations
        if not has_variation:
            product_detail_variation = {
                'inventory_id': item.get('inventory_id'),
                'currency_id': item.get('currency_id'),
                'stock': item.get('available_quantity'),
                'sold_quantity': item.get('sold_quantity'),
                'catalog_id' : item.get('catalog_product_id'),
                'seller_sku': extract_seller_sku(item.get('attributes', [])),
                'variation_id': np.nan
            }
            product_details_general.update(product_detail_variation)
            df_ = pd.DataFrame([product_details_general])
            df_product = pd.concat([df_product, df_], ignore_index=True)

        # If product has variations
        else:
            for var in item.get('variations', []):
                variation_id = var['id']
                variation = [variation for variation in content_variations if variation['id'] == variation_id][0]
                variation_id = var['id']
                product_detail_variation = {
                    'inventory_id': variation.get('inventory_id'),
                    'currency_id': variation.get('currency_id'),
                    'stock': variation.get('available_quantity'),
                    'sold_quantity': variation.get('sold_quantity'),
                    'seller_sku': extract_seller_sku(variation.get('attributes', [])),
                    'variation_id': variation_id,
                    'catalog_id' : item.get('catalog_product_id'),
                }
                product_details_general.update(product_detail_variation)
                df_ = pd.DataFrame([product_details_general])
                df_product = pd.concat([df_product, df_], ignore_index=True)

    return df_product